In [7]:
import pandas as pd
import numpy as np
import pyranges
import pickle
import os.path

In [2]:
hicLoops = pd.read_csv('./Dataset/merged_loops.bedpe',delimiter='\t')
ctcfPeaks = pyranges.read_bed('./Dataset/ENCFF356LIU.bed')
chiaPetLoops = pd.read_csv('./Dataset/4DNFIS9CCN6R.bedpe',delimiter='\t', header=None)
# chiaPetLoops= np.genfromtxt('./Dataset/4DNFIS9CCN6R.bedpe', delimiter='\t')


rad21=pyranges.read_bed('./Dataset/ENCFF834GOT.bed')

c:\Users\prusak.patryk\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyranges\methods\init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}
c:\Users\prusak.patryk\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyranges\methods\init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}


In [3]:
hicLoops=hicLoops[1:]
hicLoops.head()
# ctcfPeaks.head()

,#chr1,x1,x2,chr2,y1,y2,name,score,strand1,strand2,...,expectedH,expectedV,fdrBL,fdrDonut,fdrH,fdrV,numCollapsed,centroid1,centroid2,radius
1,10,60880000.0,60890000.0,10,60960000.0,60970000.0,.,.,.,.,...,29.857151,34.565178,5.510599e-06,3.417617e-03,6.518073e-06,4.352302e-03,1.0,60885000.0,60965000.0,0.0
2,10,131900000.0,131910000.0,10,131980000.0,131990000.0,.,.,.,.,...,47.551224,38.399870,4.084538e-06,5.531278e-03,6.420658e-03,5.203196e-06,2.0,131905000.0,131980000.0,5000.0
3,10,71720000.0,71730000.0,10,71810000.0,71820000.0,.,.,.,.,...,56.117943,43.748466,2.594937e-06,2.825991e-06,8.173968e-03,3.346481e-06,2.0,71715000.0,71815000.0,10000.0
4,10,62610000.0,62620000.0,10,62730000.0,62740000.0,.,.,.,.,...,72.266400,52.050930,6.261819e-07,7.079455e-07,5.628446e-03,7.618980e-07,1.0,62615000.0,62735000.0,0.0
5,10,26780000.0,26790000.0,10,26850000.0,26860000.0,.,.,.,.,...,65.557430,90.107475,3.840393e-07,4.753292e-07,5.291055e-07,9.047872e-03,1.0,26785000.0,26855000.0,0.0


In [4]:
hicLoopsX=hicLoops[['x1','x2']]
hicLoopsXArr=hicLoopsX.to_numpy()
ctcfPeaksStartEndArr=ctcfPeaks.df[['Start','End']].to_numpy()
rad21StartEndArr=rad21.df[['Start','End']].to_numpy()
chiaPetLoopsXArr=chiaPetLoops.to_numpy()[:,[1,2]]


In [5]:
if os.path.isfile('peakLoopHicLCtcfPPairs')==False or os.path.isfile('peakLoopHicLCtcfPSingleAnchorCount')==False:
    peakLoopHicLCtcfPPairs = []
    peakLoopHicLCtcfPSingleAnchorCount = 0
    for i,loop in enumerate(hicLoopsXArr):
        for j,peak in enumerate(ctcfPeaksStartEndArr):
            if (peak[0]>=loop[0] and peak[0]<=loop[1]) or (peak[1]>=loop[0] and peak[1]<=loop[1]):
                peakLoopHicLCtcfPSingleAnchorCount+=1
            if peak[0]>=loop[0] and peak[0]<=loop[1] and peak[1]>=loop[0] and peak[1]<=loop[1]:
                peakLoopHicLCtcfPPairs.append([i,j])

    with open("peakLoopHicLCtcfPPairs", "wb") as f:
        pickle.dump(peakLoopHicLCtcfPPairs, f)
    with open("peakLoopHicLCtcfPSingleAnchorCount", "wb") as f:
        pickle.dump(peakLoopHicLCtcfPSingleAnchorCount, f)


In [8]:
if os.path.isfile('peakLoopHicLRad21PPairs')==False or os.path.isfile('peakLoopHicLRad21PSingleAnchorCount')==False:
    peakLoopHicLRad21PPairs = []
    peakLoopHicLRad21PSingleAnchorCount = 0
    for i,loop in enumerate(hicLoopsXArr):
        for j,peak in enumerate(rad21StartEndArr):
            if (peak[0]>=loop[0] and peak[0]<=loop[1]) or (peak[1]>=loop[0] and peak[1]<=loop[1]):
                peakLoopHicLRad21PSingleAnchorCount+=1
            if peak[0]>=loop[0] and peak[0]<=loop[1] and peak[1]>=loop[0] and peak[1]<=loop[1]:
                peakLoopHicLRad21PPairs.append([i,j])


    with open("peakLoopHicLRad21PPairs", "wb") as f:
        pickle.dump(peakLoopHicLCtcfPPairs, f)
    with open("peakLoopHicLRad21PSingleAnchorCount", "wb") as f:
        pickle.dump(peakLoopHicLRad21PSingleAnchorCount, f)


In [ ]:
if os.path.isfile('peakLoopChiaLCtcfPPairs')==False or os.path.isfile('peakLoopChiaLCtcfPSingleAnchorCount')==False:
   
    peakLoopChiaLCtcfPPairs = []
    peakLoopChiaLCtcfPSingleAnchorCount = 0
    for i,loop in enumerate(chiaPetLoopsXArr):
        for j,peak in enumerate(ctcfPeaksStartEndArr):
            if (peak[0]>=loop[0] and peak[0]<=loop[1]) or (peak[1]>=loop[0] and peak[1]<=loop[1]):
                peakLoopChiaLCtcfPSingleAnchorCount+=1
            if peak[0]>=loop[0] and peak[0]<=loop[1] and peak[1]>=loop[0] and peak[1]<=loop[1]:
                peakLoopChiaLCtcfPPairs.append([i,j])


    with open("peakLoopChiaLCtcfPPairs", "wb") as f:
        pickle.dump(peakLoopChiaLCtcfPPairs, f)
    with open("peakLoopChiaLCtcfPSingleAnchorCount", "wb") as f:
        pickle.dump(peakLoopChiaLCtcfPSingleAnchorCount, f)


In [1]:
if os.path.isfile('peakLoopChiaLRad21PPairs')==False or os.path.isfile('peakLoopChiaLRad21PSingleAnchorCount')==False:

    peakLoopChiaLRad21PPairs = []
    peakLoopChiaLRad21PSingleAnchorCount = 0
    for i,loop in enumerate(chiaPetLoopsXArr):
        for j,peak in enumerate(rad21StartEndArr):
            if (peak[0]>=loop[0] and peak[0]<=loop[1]) or (peak[1]>=loop[0] and peak[1]<=loop[1]):
                peakLoopChiaLRad21PSingleAnchorCount+=1
            if peak[0]>=loop[0] and peak[0]<=loop[1] and peak[1]>=loop[0] and peak[1]<=loop[1]:
                peakLoopChiaLRad21PPairs.append([i,j])


    with open("peakLoopChiaLRad21PPairs", "wb") as f:
        pickle.dump(peakLoopChiaLRad21PPairs, f)
    with open("peakLoopChiaLRad21PSingleAnchorCount", "wb") as f:
        pickle.dump(peakLoopChiaLRad21PSingleAnchorCount, f)

NameError: name 'chiaPetLoopsXArr' is not defined

Perform 4 comparisons between datasets. for two compared datasets (A and B) calculate how many loops from A match (**) some loop from B, and how many loops from B match some loop from A. You can present the result as a 2x2 table showing counts of total and matched loops for datasets.

In [9]:
with open(r"peakLoopChiaLCtcfPPairs", "rb") as input_file: # Chia LC
     peakLoopChiaLCtcfPPairs = pickle.load(input_file)

with open(r"peakLoopHicLCtcfPPairs", "rb") as input_file: # HIC LC
     peakLoopHicLCtcfPPairs = pickle.load(input_file)

with open(r"peakLoopChiaLRad21PPairs", "rb") as input_file: # CHIA Rad
     peakLoopChiaLRad21PPairs = pickle.load(input_file)

with open(r"peakLoopHicLRad21PPairs", "rb") as input_file: # Hic Rad
     peakLoopHicLRad21PPairs = pickle.load(input_file)

chiaLcChiaRadCount = 0
singleAnchorChiaLcChiaRadCount = 0
for chiaLcIndex in peakLoopChiaLCtcfPPairs:
     for chiaLRadIndex in peakLoopChiaLRad21PPairs:
          if abs(chiaPetLoopsXArr[chiaLcIndex[0]][0]-chiaPetLoopsXArr[chiaLRadIndex[0]][0])<=1 and abs(chiaPetLoopsXArr[chiaLcIndex[0]][1]-chiaPetLoopsXArr[chiaLRadIndex[0]][1])<=1:
               chiaLcChiaRadCount+=1
          elif abs(chiaPetLoopsXArr[chiaLcIndex[0]][0]-chiaPetLoopsXArr[chiaLRadIndex[0]][0])<=1 or abs(chiaPetLoopsXArr[chiaLcIndex[0]][1]-chiaPetLoopsXArr[chiaLRadIndex[0]][1])<=1:
               singleAnchorChiaLcChiaRadCount+=1
print(chiaLcChiaRadCount)
print(singleAnchorChiaLcChiaRadCount)


chiaLcHicLcCount = 0
singleAnchorChiaLcHicLcCount = 0
for chiaLcIndex in peakLoopChiaLCtcfPPairs:
     for hicLcIndex in peakLoopChiaLCtcfPPairs:
          if abs(chiaPetLoopsXArr[chiaLcIndex[0]][0]-hicLoopsXArr[hicLcIndex[0]][0])<=1 and abs(chiaPetLoopsXArr[chiaLcIndex[0]][1]-hicLoopsXArr[hicLcIndex[0]][1])<=1:
               chiaLcHicLcCount+=1
          if abs(chiaPetLoopsXArr[chiaLcIndex[0]][0]-hicLoopsXArr[hicLcIndex[0]][0])<=1 or abs(chiaPetLoopsXArr[chiaLcIndex[0]][1]-hicLoopsXArr[hicLcIndex[0]][1])<=1:
               singleAnchorChiaLcHicLcCount+=1
print(chiaLcHicLcCount)
print(singleAnchorChiaLcHicLcCount)


chiaLcHicRadCount = 0
singleAnchorChiaLcHicRadCount = 0
for chiaLcIndex in peakLoopChiaLCtcfPPairs:
     for hicRadIndex in peakLoopHicLRad21PPairs:
          if abs(chiaPetLoopsXArr[chiaLcIndex[0]][0]-hicLoopsXArr[hicRadIndex[0]][0])<=1 and abs(chiaPetLoopsXArr[chiaLcIndex[0]][1]-hicLoopsXArr[hicRadIndex[0]][1])<=1:
               chiaLcHicRadCount+=1
          if abs(chiaPetLoopsXArr[chiaLcIndex[0]][0]-hicLoopsXArr[hicRadIndex[0]][0])<=1 or abs(chiaPetLoopsXArr[chiaLcIndex[0]][1]-hicLoopsXArr[hicRadIndex[0]][1])<=1:
               singleAnchorChiaLcHicRadCount+=1
print(chiaLcHicRadCount)
print(singleAnchorChiaLcHicRadCount)


hicLcHicRadIndex = 0
singleAnchorHicLcHicRadIndex = 0
for hicLcIndex in peakLoopHicLCtcfPPairs:
     for hicRadIndex in peakLoopHicLRad21PPairs:
          if abs(hicLoopsXArr[hicLcIndex[0]][0]-hicLoopsXArr[hicRadIndex[0]][0])<=1 and abs(hicLoopsXArr[hicLcIndex[0]][1]-hicLoopsXArr[hicRadIndex[0]][1])<=1:
               hicLcHicRadIndex+=1
          if abs(hicLoopsXArr[hicLcIndex[0]][0]-hicLoopsXArr[hicRadIndex[0]][0])<=1 or abs(hicLoopsXArr[hicLcIndex[0]][1]-hicLoopsXArr[hicRadIndex[0]][1])<=1:
               singleAnchorHicLcHicRadIndex+=1
print(hicLcHicRadIndex)
print(singleAnchorHicLcHicRadIndex)

FileNotFoundError: [Errno 2] No such file or directory: 'peakLoopChiaLCtcfPPairs'

Make a visualization of a selected region (e.g. in IGV), choose something that looks nicely, zoom in enough so that you can see the peak locations. You can also include the loops matched between 2 datasets.